In [1]:
from architectures import *
from dataloader import iqa_dataset
from config import *
from torch.utils.data import DataLoader
from torchvision import transforms

In [2]:
batch_size = 1

train_data=iqa_dataset(part='train',labels_path='../Databases/tid2013/scores.pickle' ,db_path='../Databases/tid2013/Images/',
                 ids_path='../Databases/tid2013/IDs.pickle',
                       transform=transforms.Compose([transforms.ToTensor()]))

val_data = iqa_dataset(part='test',labels_path='../Databases/tid2013/scores.pickle' ,db_path='../Databases/tid2013/Images/',
                 ids_path='../Databases/tid2013/IDs.pickle',
                       transform=transforms.Compose([transforms.ToTensor()]))

print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

train_dl = DataLoader(train_data, batch_size, shuffle = True)
val_dl = DataLoader(val_data,1,shuffle=False)


Length of Train Data : 2400
Length of Validation Data : 600


In [3]:
# Initialize the model for this run
model = initialize_model('vgg16',True,True)
model=FC(model,2,1024,0.25,'vgg16')
model.to(device)
model.classifier.load_state_dict(torch.load('../pretrainedModels/vgg16_tid2013.pth', map_location = device))
model = model.eval()

/home/hbrachem@admr.insa/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hbrachem@admr.insa/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
from attacks.fgm import fast_gradient_method 

from tqdm import tqdm
import numpy as np

In [5]:
#Tensorboard

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/vgg16_tid2013")

2023-01-17 08:53:03.569349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 08:53:03.669165: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-17 08:53:04.028386: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-17 08:53:04.028423: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [7]:
%reload_ext tensorboard

In [8]:
LOG_DIR = 'runs/'

get_ipython().system_raw(
    'tensorboard --logdir {} --host localhost --port 8088 &'
    .format(LOG_DIR)
)

2023-01-17 08:53:05.600903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 08:53:05.692696: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-17 08:53:05.694875: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-17 08:53:05.694884: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [9]:
epsilon = 0.001
iterations = 5
noise_list = []
x_adv_list = []

attack = "fgm"
loss = '1-max'




for i ,[im, label] in enumerate(tqdm(val_dl)):
    im = im.to(device)
    if attack == "fgm":
        x_adv_list.append(fast_gradient_method(model,im, epsilon,np.inf,y=torch.tensor([[10]]).float()))
    if attack == "pgd":
        x_adv_list.append(projected_gradient_descent(model,im,epsilon, 0.01,iterations, np.inf))
    if attack == "cw":
        x_adv_list.append(carlini_wagner_l2(model,im,1))
    if attack == "hsj":
        x_adv_list.append(hop_skip_jump_attack(model,im,1))
    if attack == "sL1":
        x_adv_list.append(sparse_l1_descent(model,im,epsilon))
    if attack == "spsa":
        x_adv_list.append(spsa(model,im,epsilon,iterations,np.inf))
    if attack == "bim":
        x0 = torch.clone(im)
        for j in range(iterations):
            x0 = fast_gradient_method(model,x0, epsilon, np.inf)
        x_adv_list.append(x0)
    
    
    noise_list.append(x_adv_list[i]-im)
    if i%100 == 0:
        writer.add_image("Images_"+attack+"_"+loss+"/x_adv_"+str(epsilon),x_adv_list[i][0],i)
        writer.add_image("Images/x_orig",im[0],i)
        writer.add_image("Images_"+attack+"_"+loss+"/noise_"+str(epsilon),x_adv_list[i][0]-im[0],i)



  2%|▉                                         | 13/600 [00:08<06:40,  1.46it/s]


KeyboardInterrupt: 

In [10]:
import matplotlib.pyplot as plt

In [ ]:
epsilon = 0.001
out = '../samples/vgg16/tid2013/fgm/mse/epsilon='+str(epsilon)
!mkdir $out
attack = "fgm"

for i ,[im, label] in enumerate(tqdm(val_dl)):
    if i % 2 == 1:
        continue
    im = im.to(device)
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(28,32))
    ax1.axis('off')
    ax1.xaxis.tick_top()
    ax2.axis('off')
    ax2.xaxis.tick_top()
    x_adv = fast_gradient_method(model,im, epsilon,np.inf,y=label.float())#torch.tensor([[10]]).float())
    ax1.set_title('x_adv',fontsize=40)
    ax2.set_title('x_orig',fontsize=40)
    ax1.imshow(x_adv[0].permute(1, 2, 0).detach().numpy())
    ax2.imshow(im[0].permute(1, 2, 0).detach().numpy())
    fig.savefig(out+'/'+str(i),bbox_inches='tight')
    if i > 50:
        break

  2%|▉                                         | 13/600 [00:09<06:38,  1.47it/s]

In [20]:
im.shape

torch.Size([1, 3, 384, 512])

In [13]:
from eval_funct import *

In [12]:
SRCC,KRCC,PLCC,RMSE,fr,mos = evaluate(val_dl,model,normalize_imagenet, None ,None)

100%|█████████████████████████████████████████| 600/600 [03:03<00:00,  3.26it/s]


In [13]:
print("SRCC: ",SRCC)
print("PLCC: ",PLCC)
print("KRCC: ",KRCC)
print("RMSE: ",RMSE)

SRCC:  0.8805628318478979
PLCC:  0.9096687653736553
KRCC:  0.6974983938087685
RMSE:  0.4322383376612332


In [22]:
SRCC,KRCC,PLCC,RMSE,fr,mos = evaluate(val_dl,model,normalize_imagenet,noise_list,None)

100%|█████████████████████████████████████████| 600/600 [02:53<00:00,  3.46it/s]


In [23]:
print("SRCC: ",SRCC)
print("PLCC: ",PLCC)
print("KRCC: ",KRCC)
print("RMSE: ",RMSE)
print("fr: ",fr)

SRCC:  0.09445653904824962
PLCC:  0.3435235258434083
KRCC:  0.054680656337394905
RMSE:  1.1202166317609985
fr:  []


In [21]:
max(mos)

7.1